In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd



In [ ]:

# Load the Excel file
excel_path = "witcher_info.xlsx"  # Change this to your actual file path
xls = pd.ExcelFile(excel_path)

# List all sheet names
print(xls.sheet_names)

# Load a specific sheet (e.g., 'Gounter O'Dim')
df = pd.read_excel(xls, sheet_name="Gounter O'Dim")

# Display the first few rows
print(df.head())

# Access specific columns (e.g., 'Story', 'Scene')
selected_columns = df[['Story', 'Scene', 'Participants', 'Dialogue Game', 'Dialogue LLM']].copy()
print(selected_columns)

selected_columns.columns = ['Story', 'Scene', 'Participants', 'Dialogue Game', 'Dialogue LLM']



# Define the texts+
Story = selected_columns['Story'].get(0)
Scene = selected_columns['Scene'].get(0)
Dialogue_Game = selected_columns['Dialogue Game'].get(0)
Dialogue_LLM = selected_columns['Dialogue LLM'].get(0)
# Shared participants
# participants = ["Geralt of Rivia was a legendary witcher of the School of the Wolf active throughout the 13th century. He loved the sorceress Yennefer, considered the love of his life despite their tumultuous relationship, and became Ciri's adoptive father.During the Trial of the Grasses, Geralt exhibited unusual tolerance for the mutagens that grant witchers their abilities. Accordingly, Geralt was subjected to further experimental mutagens which rendered his hair white and may have given him greater speed, strength, and stamina than his fellow witchers.Despite his title, Geralt did not hail from the city of Rivia. After being left with the witchers by his mother, Visenna, he grew up in their keep of Kaer Morhen in the realm of Kaedwen. In the interest of appearing more trustworthy to potential clients, young witchers were encouraged to make up surnames for themselves by master Vesemir. As his first choice, Geralt chose Geralt Roger Eric du Haute-Bellegarde, but this choice was dismissed by Vesemir as silly and pretentious, so Geralt was all that remained of his chosen name. Of Rivia was a more practical alternative and Geralt even went so far as to adopt a Rivian accent to appear more authentic. Later, Queen Meve of Lyria knighted him for his valor in the Battle for the Bridge on the Yaruga conferring on him the formal title of Rivia, which amused him.[2] He, therefore, became a true knight.", 
#                 "Peter Saar Gwynleve was the captain of the Nilfgaardian Garrison located in White Orchard. By all appearances he seemed a rather fair and kind person to the people of White Orchard, especially by the standards of most conquering Nilfgaardians."]
# Participants_Text = " ".join(participants)
participants = selected_columns['Participants']
Participants_Text = "" 
for i in range(len(participants)):
    Participants_Text.join(str(participants[i]))

texts = [Story, Scene, Dialogue_Game, Dialogue_LLM, Participants_Text]
print(f"Participants: {Participants_Text}")

['NilfgardOficer', "Gounter O'Dim", 'Ghoul']
   Unnamed: 0                                              Story  \
0         NaN  After rescuing the merchant Bram from the clut...   
1         NaN                                                NaN   

                                               Scene  \
0  Geralt is looking for Yennefer and encounters ...   
1                                                NaN   

                                        Participants  \
0  Geralt of Rivia was a legendary witcher of the...   
1  Gaunter O'Dimm, sometimes called Master Mirror...   

                                       Dialogue Game  \
0  Geralt: Looking for a woman.\nGaunter: Ahh, li...   
1                                                NaN   

                                        Dialogue LLM  \
0  Gaunter: You look lost, witcher. Or... perhaps...   
1                                                NaN   

                                          Unnamed: 6  
0  Cosine Similarit

In [17]:
import chromadb
import os
from yake import KeywordExtractor


kw_extractor = KeywordExtractor(top=5, stopwords=None, n=3, dedupLim=0.9, features=None)

script_dir = os.getcwd()
chroma_client = chromadb.PersistentClient(path=script_dir)
collection = chroma_client.get_or_create_collection(name="witcher")

keywords_game = kw_extractor.extract_keywords(Dialogue_Game)
keywords_llm = kw_extractor.extract_keywords(Dialogue_LLM)


def get_keywords_similarity(keywords, main_text):
    texts = [main_text]
    for keyword in keywords:
        # Add the keyword to the ChromaDB collection
        keyword_result = collection.query(
            query_texts=keyword[0], # Chroma will embed this for you
            n_results=1 # how many results to return
        )
        texts.append(str(keyword_result['documents'][0]))

    vectorizer = TfidfVectorizer()
    print(f"Texts: { texts}")
    tfidf_matrix = vectorizer.fit_transform(texts)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    avg_similarity = similarity_matrix[0][1:]
    avg_similarity = sum(avg_similarity) / len(avg_similarity)
    return avg_similarity
    

game_keywards_similarity = get_keywords_similarity(keywords_game, Dialogue_Game)
llm_keywards_similarity = get_keywords_similarity(keywords_llm, Dialogue_LLM)


Texts: ["Geralt: Looking for a woman.\nGaunter: Ahh, like everyone.\nGeralt: Not like everyone, and not just any woman. Mine smells of lilac and gooseberries, dresses in black and white.\nGaunter: Two schnappses!\nGaunter: Hah. It'll lift your spirits.\nGeralt: Fine, I'll have a drink.\nGeralt: Can we cut to the chase? You seen her or not?\nGaunter: Yennefer of Vengerberg?\nGeralt: Never mentioned her name.\nGaunter: Yet you described her perfectly. And once I hear something, I never forget. Can't help it.\nGeralt: You seen Yennefer?\nGaunter: Deepest apologies, but I must ask: Is this about love?\nGeralt: None of your business.\nGaunter: Yes, as a vagrant I deserve no explanation.\nGeralt: What do you know? Tell me.\nGaunter: Before you appeared, it never occurred to me that might've been Yennefer. Who would've have thought…\nGeralt: Get to the point.\nGaunter: A Nilfgaardian scout from the local garrison saw her.\nGeralt: Where?\nGaunter: At their camp. She rode in there - dark of ni

In [20]:
# TF-IDF vectorization
from chromadb.config import Settings


vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)


# Indices
i_story = 0
i_scene = 1
i_game = 2
i_llm = 3
i_participants = 4

# Cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

# Cosine similarities
print("Cosine Similarity Results:\n")

print(f"Story           ↔ Dialogue_Game : {similarity_matrix[i_story][i_game]:.4f}")
print(f"Story           ↔ Dialogue_LLM  : {similarity_matrix[i_story][i_llm]:.4f}")
print(f"Scene           ↔ Dialogue_Game : {similarity_matrix[i_scene][i_game]:.4f}")
print(f"Scene           ↔ Dialogue_LLM  : {similarity_matrix[i_scene][i_llm]:.4f}")
print(f"Dialogue_Game   ↔ Dialogue_LLM  : {similarity_matrix[i_game][i_llm]:.4f}")
print(f"Participants    ↔ Dialogue_Game : {similarity_matrix[i_participants][i_game]:.4f}")
print(f"Participants    ↔ Dialogue_LLM  : {similarity_matrix[i_participants][i_llm]:.4f}")
print(f"Game Keywords Similarity: {game_keywards_similarity:.4f}")
print(f"LLM Keywords Similarity: {llm_keywards_similarity:.4f}\n")

# Summary
print("Summary of Cosine Similarities:\n")
print(f"Story: G-{similarity_matrix[i_story][i_game]:.4f} LLM-{similarity_matrix[i_story][i_llm]:.4f}")
print(f"Scene: G-{similarity_matrix[i_scene][i_game]:.4f} LLM-{similarity_matrix[i_scene][i_llm]:.4f}")
print(f"Participants: G-{similarity_matrix[i_participants][i_game]:.4f} LLM-{similarity_matrix[i_participants][i_llm]:.4f}")
print(f"Keywords Similarity: G-{game_keywards_similarity:.4f} LLM-{llm_keywards_similarity:.4f}\n")

Cosine Similarity Results:

Story           ↔ Dialogue_Game : 0.3294
Story           ↔ Dialogue_LLM  : 0.2945
Scene           ↔ Dialogue_Game : 0.4167
Scene           ↔ Dialogue_LLM  : 0.3063
Dialogue_Game   ↔ Dialogue_LLM  : 0.5793
Participants    ↔ Dialogue_Game : 0.0000
Participants    ↔ Dialogue_LLM  : 0.0000
Game Keywords Similarity: 0.1531
LLM Keywords Similarity: 0.1015

Summary of Cosine Similarities:

Story: G-0.3294 LLM-0.2945
Scene: G-0.4167 LLM-0.3063
Participants: G-0.0000 LLM-0.0000
Keywords Similarity: G-0.1531 LLM-0.1015

